In [11]:
import pandas as pd 
import numpy as np

In [12]:
basefile = pd.read_csv("data/2020_Claims_SingleSource_v1.4.csv",dtype=str)
newfile = pd.read_csv("data/2021_Claims_SingleSource_v1.3.csv",dtype=str)

basefile.columns = basefile.columns.str.replace(' ', '_')
newfile.columns = newfile.columns.str.replace(' ', '_')
# Version to capture 
basefile["VERSION"] = "BASE"
newfile["VERSION"] = "NEW"
# Const to capture the whole file
basefile["CONST"]="CONST"
newfile["CONST"]="CONST"

### Count comparison

In [13]:
basefile.count()

Measure_ID           19434
DATA_ELEMENT_NAME    19434
CODING_SYSTEM        19434
CODE                 19434
MODIFIER               883
PLACE_OF_SERVICE       724
AGE                  19434
GENDER               19434
VERSION              19434
CONST                19434
dtype: int64

In [14]:
newfile.count()

Measure_ID           21380
DATA_ELEMENT_NAME    21380
CODING_SYSTEM        21380
CODE                 21380
MODIFIER               370
PLACE_OF_SERVICE       355
AGE                  21380
GENDER               21380
VERSION              21380
CONST                21380
dtype: int64

### Joining the two files 

In [15]:
key=newfile.columns.values.tolist()

In [16]:
key.remove("VERSION")

In [17]:
joint=newfile.merge(basefile,on=key,how="outer",suffixes=["_new","_base"])

In [18]:
def getJoinMeta(row):
    val = "NONE"
    if (row["VERSION_new"] == "NEW") & (row ["VERSION_base"] == "BASE"):
        val = "COMMON"
    elif row["VERSION_new"] == "NEW":
        val = "NEW"
    else: 
        val = "BASE" 
    return val

In [19]:
joint["VERSION"]= joint.apply(getJoinMeta,axis=1)
joint=joint.drop(["VERSION_new","VERSION_base"],axis=1)

In [20]:
joint[joint.VERSION=="NONE"]

,Measure_ID,DATA_ELEMENT_NAME,CODING_SYSTEM,CODE,MODIFIER,PLACE_OF_SERVICE,AGE,GENDER,CONST,VERSION


In [21]:
joint

,Measure_ID,DATA_ELEMENT_NAME,CODING_SYSTEM,CODE,MODIFIER,PLACE_OF_SERVICE,AGE,GENDER,CONST,VERSION
0,1,ENCOUNTER_CODE,C4,99202,NaN,NaN,18 - 75,"M, F",CONST,COMMON
1,1,ENCOUNTER_CODE,C4,99203,NaN,NaN,18 - 75,"M, F",CONST,COMMON
2,1,ENCOUNTER_CODE,C4,99204,NaN,NaN,18 - 75,"M, F",CONST,COMMON
3,1,ENCOUNTER_CODE,C4,99205,NaN,NaN,18 - 75,"M, F",CONST,COMMON
4,1,ENCOUNTER_CODE,C4,99212,NaN,NaN,18 - 75,"M, F",CONST,COMMON
...,...,...,...,...,...,...,...,...,...,...
25968,437,G_CODE_PD,HCPCS,G9641,NaN,NaN,≥0,"M, F",CONST,BASE
25969,437,G_CODE_PN,HCPCS,G9641,NaN,NaN,≥0,"M, F",CONST,BASE
25970,437,G_CODE_PD_Exl,HCPCS,G9640,NaN,NaN,≥0,"M, F",CONST,BASE
25971,437,G_CODE_PD,HCPCS,G9639,NaN,NaN,≥0,"M, F",CONST,BASE


In [22]:
onlybase=joint[joint.VERSION=="BASE"]
len(onlybase)

4560

In [23]:
onlynew=joint[joint.VERSION=="NEW"]
len(onlynew)

6513

In [24]:
intersection=joint[joint.VERSION=="COMMON"]

In [25]:
len(intersection)

14900

# New against Base comparison

## Defining functions

In [26]:
def columnToSet(df,column:str):
    return set(df[column].unique().tolist())

In [48]:
def analyze_difference(joint,partcolumn,partid,subcol,debug=False):
    subset=joint[joint[partcolumn]==partid]
    basedf=subset[(subset.VERSION=="BASE") | (subset.VERSION=="COMMON")]
    newdf=subset[(subset.VERSION=="NEW")|(subset.VERSION=="COMMON")]
    
    newids=columnToSet(newdf,subcol)
    baseids=columnToSet(basedf,subcol)
    
    added=newids-baseids
    removed=baseids-newids
    
    #get all the changed IDs 
    changedids=columnToSet(subset[subset.VERSION!="COMMON"],subcol)
    
    
    changed=changedids-added-removed
    
    if debug==True:
        print("Added")
        print(added)
        print("Removed")
        print(removed)
        print("Changed")
        print(changed)
    
    return (subset,added|removed|changed,added,removed,changed)
    

In [28]:
(df,changed)=analyze_difference(joint,"Measure_ID","93","CODE")

### Iterative process

In [29]:
(df,changed)=analyze_difference(joint,"CONST","CONST","Measure_ID")

In [30]:
(dfL1,changed)=analyze_difference(df,"Measure_ID","93","DATA_ELEMENT_NAME")

In [31]:
(dfL2,changed)=analyze_difference(dfL1,"DATA_ELEMENT_NAME","CPT_II_PD_Exe","CODE")

In [32]:
dfL2[dfL2.CODE=="4131F"]

,Measure_ID,DATA_ELEMENT_NAME,CODING_SYSTEM,CODE,MODIFIER,PLACE_OF_SERVICE,AGE,GENDER,CONST,VERSION
9983,93,CPT_II_PD_Exe,CPT_II,4131F,1P,NaN,≥2,"M, F",CONST,NEW
23101,93,CPT_II_PD_Exe,CPT_II,4131F,"1P, ≠ 2P, 3P, 8P",NaN,≥2,"M, F",CONST,BASE


In [33]:
### Recursion

In [60]:
(df,modified_mid,added_mid,removed_mid,changed_mid)=analyze_difference(joint,"CONST","CONST","Measure_ID")

In [61]:
mid={}

modified_den={}
added_den={}
changed_den={}
removed_den={}

with open("csv_report/Summary.md","a") as f:
    print("# Measure ID Summary",file=f)
    print("*Added Measures*:"+str(added_mid),file=f)
    print("*Removed Measures*:"+str(removed_mid),file=f)
    print("*Changed Measures*:"+str(changed_mid),file=f)

for i in changed_mid:
    with open("csv_report/Measure"+i+".md","a") as f:
        print("# Comparison for Measure ID "+i+"= ",file=f)
        (mid[i],modified_den[i],added_den[i],removed_den[i],changed_den[i])=analyze_difference(df,"Measure_ID",i,"DATA_ELEMENT_NAME",False)
        den = {}
        modified_code = {}
        added_code={}
        removed_code={}
        changed_code={}

        print("Data Element Name Summary",file=f)
        print("*Added DEN*:"+str(added_den[i]),file=f)
        print("*Removed DEN*:"+str(removed_den[i]),file=f)
        print("*Changed DEN*:"+str(changed_den[i]),file=f)

        for j in modified_den[i]:
            print("## For Data Element name "+j,file=f)
            (den[j],modified_code[j],added_code[j],removed_code[j],changed_code[j]) = analyze_difference(mid[i],"DATA_ELEMENT_NAME",j,"CODE",False)

            print("Codes Summary",file=f)
            print("* Number of Added CODES*:"+str(len(added_code[j])),file=f)
            print("* Number of Removed CODES*:"+str(len(removed_code[j])),file=f)
            print("* Number of Changed CODES*:"+str(len(changed_code[j])),file=f)

            if(len(added_code[j])>0):
                print("### Added Codes",file=f)
                print(den[j][den[j]["CODE"].isin(added_code[j])].to_markdown(tablefmt="grid"),file=f)
            if(len(removed_code[j])>0):
                print("### removed Codes",file=f)
                print(den[j][den[j]["CODE"].isin(removed_code[j])].to_markdown(tablefmt="grid"),file=f)
            if(len(changed_code[j])>0):
                print("### modified Codes",file=f)
                print(den[j][den[j]["CODE"].isin(modified_code[j])].to_markdown(tablefmt="grid"),file=f)
        break

In [42]:
!mkdir csv_report

In [49]:
!rm csv_report/226.01.md